## Week Three Project Example: Do not use this code example Only!!!!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np


In [2]:
# Load the CSV file
input_file = 'extracted_honeypot_logs.csv'  # Replace with your actual CSV file name
df = pd.read_csv(input_file)


In [3]:
# Ensure the timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])


KeyError: 'timestamp'

In [ ]:
# Extract useful time-based features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['day_of_month'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month


In [ ]:
# Encode the source IP addresses to numeric values using the correct column name
le = LabelEncoder()
df['src_ip_encoded'] = le.fit_transform(df['src_ip'])


In [ ]:
# Create a target variable: predict whether an IP will appear in the next hour
df['target'] = df['src_ip_encoded'].shift(-1)
df = df.dropna()  # Drop the last row with NaN target


In [ ]:
# Define the features and target
X = df[['hour', 'day_of_week', 'day_of_month', 'month', 'src_ip_encoded']]
y = df['target']


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


In [ ]:
# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


In [ ]:
# Predict on the test set
y_pred = clf.predict(X_test)


In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
# Example of predicting the next source IP address using an IP from the training data
sample_ip = df['src_ip'].iloc[0]  # Use the first IP in the dataset for prediction
encoded_ip = le.transform([sample_ip])[0]

# Create a DataFrame for the sample to match the expected feature format
sample = pd.DataFrame({
    'hour': [14],
    'day_of_week': [3],
    'day_of_month': [15],
    'month': [8],
    'src_ip_encoded': [encoded_ip]
})

# Predict the next IP address
predicted_ip_encoded = clf.predict(sample)


In [ ]:
# Ensure the predicted value is within the valid range
predicted_ip_encoded = predicted_ip_encoded.astype(int)  # Convert to integer type if needed
if 0 <= predicted_ip_encoded[0] < len(le.classes_):
    predicted_ip = le.inverse_transform(predicted_ip_encoded)
    print(f'Predicted next IP address: {predicted_ip[0]}')
else:
    print("Predicted IP address is outside the known range of IPs.")
